<a href="https://colab.research.google.com/github/lanukahjo/Image-Splicing-Localisation/blob/main/Image_Splicing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Image Splicing Localisation
#### This notebook is the main flow of execution

In [507]:
from skimage import data
from skimage import filters
from skimage import io
from skimage.feature import greycomatrix
import numpy as np
import os

Reading the file from the given location
Manually for now....

In [518]:
PATH_TO_AUTHENTIC_IMG = os.getcwd() + '/ImSpliceDataset/Au-T/AU_T_023.bmp'
PATH_TO_SPLICED_IMG = os.getcwd() + '/ImSpliceDataset/Sp-T/SP_T_023.bmp';
THRESHOLD = 0.25;
BLOCK_SIZE = 8;
STEP_SIZE = 2;

###Assumption
- Images are already in grayscale format (0-255)


In [519]:
authentic_img = io.imread(PATH_TO_AUTHENTIC_IMG);
spliced_img = io.imread(PATH_TO_SPLICED_IMG); 

In [520]:
output_img_euclidean_dist = spliced_img;
output_img_hellinger_dist = spliced_img;

authentic_img_grey_reduced = (authentic_img/32).astype(int);
spliced_img_grey_reduced = (spliced_img/32).astype(int);

img_height = authentic_img.shape[0]
img_width = authentic_img.shape[1]

In [521]:
# Get Hellinger Distance between two numpy ndarrays x and y
def getHellingerDistance(x, y):
  dot_product = np.multiply(x,y);
  squared_matrix = x - 2*np.sqrt(dot_product) + y;
  hellinger_distance = math.sqrt(np.sum(squared_matrix)/2.0); 
  return hellinger_distance;

In [522]:
# Get euclidean distance between two numpy ndarrays x and y
def getEuclideanDistance(x, y):
  euclidean_distance = math.sqrt(np.sum(np.square(x-y)))
  return euclidean_distance; 

In [523]:
# We shall store the euclidean and hellinger distances because we need to normalise the diatances too....
dist_mat_height = (int)((img_height - BLOCK_SIZE)/STEP_SIZE + 1); #total number of blocks in a column
dist_mat_width = (int)((img_width - BLOCK_SIZE)/STEP_SIZE + 1); #total number of blocks in a row
euc_dist_mat = np.zeros((dist_mat_height,dist_mat_width),dtype=float) #creating initial empty arrays
hell_dist_mat = np.zeros((dist_mat_height,dist_mat_width),dtype=float) 

In [524]:
row = 0; 
col = 0; #these two variables are used to access the euclidean and hellinger distance matrices
for i in range(BLOCK_SIZE-1,img_height,STEP_SIZE):
  col = 0;
  
  for j in range(BLOCK_SIZE-1,img_width,STEP_SIZE):
    authentic_block = authentic_img_grey_reduced[i+1-BLOCK_SIZE:i+1, j+1-BLOCK_SIZE:j+1]
    spliced_block = spliced_img_grey_reduced[i+1-BLOCK_SIZE:i+1, j+1-BLOCK_SIZE:j+1]
    
    glcm_matrix_auth = greycomatrix(authentic_block,distances=[1],angles=[0],levels = 8,normed=True)[:,:,0,0]
    glcm_matrix_spl = greycomatrix(spliced_block,distances=[1],angles=[0],levels = 8,normed=True)[:,:,0,0]
  
    hellinger_distance = getHellingerDistance(glcm_matrix_auth, glcm_matrix_spl);
    euclidean_distance = getEuclideanDistance(glcm_matrix_auth, glcm_matrix_spl);

    hell_dist_mat[row,col] = hellinger_distance
    euc_dist_mat[row,col] = euclidean_distance

    col = col + 1
  row = row + 1

In [525]:
euc_dist_mat = euc_dist_mat / np.max(euc_dist_mat)
hell_dist_mat = hell_dist_mat / np.max(hell_dist_mat)

In [526]:
#The blocks for which the textural difference is greater than threshold is painted white....
for i in range(0,dist_mat_height):
    for j in range(0,dist_mat_width):
        if (euc_dist_mat[i,j] > THRESHOLD):
            output_img_euclidean_dist[i*STEP_SIZE:i*STEP_SIZE+BLOCK_SIZE, j*STEP_SIZE:j*STEP_SIZE+BLOCK_SIZE] = 255*np.ones((BLOCK_SIZE,BLOCK_SIZE))
        if (hell_dist_mat[i,j] > THRESHOLD):
            output_img_hellinger_dist[i*STEP_SIZE:i*STEP_SIZE+BLOCK_SIZE, j*STEP_SIZE:j*STEP_SIZE+BLOCK_SIZE] = 255*np.ones((BLOCK_SIZE,BLOCK_SIZE))


In [527]:
pos1 = PATH_TO_SPLICED_IMG.rindex("/")
pos2 = PATH_TO_SPLICED_IMG.rindex(".") #the text between the last '/' and the last '.' is the file name
base_file_name = PATH_TO_SPLICED_IMG[pos1:pos2]
io.imsave('output/Sp-T' + base_file_name + '_euclidean.bmp', output_img_euclidean_dist)
io.imsave('output/Sp-T' + base_file_name + '_hellinger.bmp', output_img_hellinger_dist)